In [1]:
import pandas as pd

In [28]:
# 전체 유저 

# reward_test_all_user.csv 산출 쿼리
# select date_format(u.join_date, '%Y-%m-%d') as join_date, u.id as uid
# from user u
# where u.join_date between '2019-10-21 00:00:00' and '2019-10-23 59:59:59'


read_path = '../csv/reward_test_all_user.csv'
all_user = pd.read_csv(read_path)

all_user['uid'] = all_user['uid'].astype(str)

all_user.tail()

,join_date,uid
19490,2019-10-23,10117330
19491,2019-10-23,10117331
19492,2019-10-23,10117332
19493,2019-10-23,10117333
19494,2019-10-23,10117334


In [30]:
# 이벤트 전송된 유저
read_path = '../csv/reward_test_target_user.csv'
target_user = pd.read_csv(read_path)

target_user = target_user[target_user['전송 실패'] != '전송 실패'] # 번개톡 전송 실패 제외

target_user['event_date'] = target_user['date']

target_user = target_user[['event_date', 'uid', 'test_group']]

target_user['uid'] = target_user['uid'].astype(str)


target_user.tail()

,event_date,uid,test_group
5337,2019-10-24,10117321,group_1
5338,2019-10-24,10117322,group_2
5339,2019-10-24,10117329,group_1
5340,2019-10-24,10117330,group_2
5341,2019-10-24,10117332,group_0


In [31]:
# 전체 유저와 이벤트 유저 merge

user = pd.merge(all_user, target_user, on='uid', how='left')

def group_reward(row):
    if row['test_group'] == 'group_0':
        return '2000'
    elif row['test_group'] == 'group_1':
        return '4000'
    elif row['test_group'] == 'group_2':
        return '6000'
    elif row['test_group'] == 'group_3':
        return '8000'
    else:
        return '0'

user['reward'] = user.apply(group_reward, axis=1)
    
user

,join_date,uid,event_date,test_group,reward
0,2019-10-21,10096134,NaN,NaN,0
1,2019-10-21,10096135,2019-10-22,group_3,8000
2,2019-10-21,10096136,2019-10-22,group_0,2000
3,2019-10-21,10096137,NaN,NaN,0
4,2019-10-21,10096138,2019-10-22,group_2,6000
...,...,...,...,...,...
19490,2019-10-23,10117330,2019-10-24,group_2,6000
19491,2019-10-23,10117331,NaN,NaN,0
19492,2019-10-23,10117332,2019-10-24,group_0,2000
19493,2019-10-23,10117333,NaN,NaN,0


In [26]:
# 이벤트 기간에 등록된 상품


# reward_test_registered_product_191021_191027.csv 산출 쿼리
# select u.uid, u.event_group, p.create_date
# from (
#     select distinct u.id as uid,
#         case mod(cast(u.id as unsigned), 4)
#             when 0 then 'group_0_completed'
#             when 1 then 'group_1_completed'
#             when 2 then 'group_2_completed'
#             when 3 then 'group_3_completed'
#         end as event_group
#     from user u
#     where u.status = 0 # 탈퇴 유저 제외
#         and u.join_date between '2019-10-21 00:00:00' and '2019-10-23 23:59:59' # 이벤트 대상 가입일
# ) u
# join product_info p
# on u.uid = p.uid
# where p.create_date between '2019-10-22 00:00:00' and '2019-10-31 23:59:59' # 상품 등록 기간





read_path = '../csv/reward_test_registered_product_191021_191027.csv'
product_raw = pd.read_csv(read_path)

product_uid = product_raw.groupby(['uid'], as_index=False).agg({'pid': 'count'})

product_uid['uid'] = product_uid['uid'].astype(str)

product_uid

,uid,pid
0,192,2
1,215,1
2,234,2
3,321,2
4,405,1
...,...,...
102199,10138821,1
102200,10139911,1
102201,10141538,1
102202,10141869,1


In [47]:
user_product_uid = pd.merge(user, product_uid, on='uid', how='left')

# user_product_uid['pid'] = user_product_uid['pid'].fillna(0)

user_product_uid

,join_date,uid,event_date,test_group,reward,pid
0,2019-10-21,10096134,NaN,NaN,0,NaN
1,2019-10-21,10096135,2019-10-22,group_3,8000,NaN
2,2019-10-21,10096136,2019-10-22,group_0,2000,NaN
3,2019-10-21,10096137,NaN,NaN,0,NaN
4,2019-10-21,10096138,2019-10-22,group_2,6000,NaN
...,...,...,...,...,...,...
19490,2019-10-23,10117330,2019-10-24,group_2,6000,NaN
19491,2019-10-23,10117331,NaN,NaN,0,NaN
19492,2019-10-23,10117332,2019-10-24,group_0,2000,NaN
19493,2019-10-23,10117333,NaN,NaN,0,NaN


In [36]:
product_uid__ = user_product_uid.groupby(['join_date'], as_index=False).agg({'pid': 'sum'})
product_uid__

,join_date,pid
0,2019-10-21,1719.0
1,2019-10-22,1637.0
2,2019-10-23,1342.0


In [48]:
product_group = user_product_uid.groupby(['join_date', 
                                          'reward'], as_index=False).agg({'uid':'count', 
                                                                          'pid':['count', 'sum']})

product_group

join_date reward   uid   pid       
                      count count    sum
0   2019-10-21      0  4924   323  846.0
1   2019-10-21   2000   453    60  154.0
2   2019-10-21   4000   475    75  271.0
3   2019-10-21   6000   441    68  195.0
4   2019-10-21   8000   511    78  253.0
5   2019-10-22      0  4590   291  891.0
6   2019-10-22   2000   410    44  167.0
7   2019-10-22   4000   411    67  226.0
8   2019-10-22   6000   435    54  209.0
9   2019-10-22   8000   452    64  144.0
10  2019-10-23      0  4674   295  703.0
11  2019-10-23   2000   455    53  141.0
12  2019-10-23   4000   435    75  178.0
13  2019-10-23   6000   425    53  164.0
14  2019-10-23   8000   404    62  156.0

In [ ]:

select u.uid, u.event_group, p.create_date
from (
    select distinct u.id as uid,
        case mod(cast(u.id as unsigned), 4)
            when 0 then 'group_0_completed'
            when 1 then 'group_1_completed'
            when 2 then 'group_2_completed'
            when 3 then 'group_3_completed'
        end as event_group
    from user u
    where u.status = 0 # 탈퇴 유저 제외
        and u.join_date between '2019-10-21 00:00:00' and '2019-10-23 59:59:59' # 이벤트 대상 가입일
) u
join product_info p
on u.uid = p.uid
where p.create_date between '2019-10-22 00:00:00' and '2019-10-31 59:59:59' # 상품 등록 기간






    select distinct u.id
    from user u
    where u.status = 0 # 탈퇴 유저 제외
        and u.join_date between '2019-10-21 00:00:00' and '2019-10-23 59:59:59' # 이벤트 대상 가입일